In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Step 1: Load the MNIST dataset
def load_data():
    mnist = datasets.fetch_openml("mnist_784", version=1, as_frame=False)
    X, y = mnist.data, mnist.target
    y = y.astype(np.int32)  # Convert target to integers
    return X, y


# Step 2: Split the data into training and test sets
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


def avg_pooling(image, pool_size):
    """
    Perform average pooling on an image.

    Parameters:
    - image: 2D numpy array representing the image.
    - pool_size: Tuple (pool_height, pool_width) for the pooling region.

    Returns:
    - pooled_image: 2D numpy array after applying average pooling.
    """
    pooled_height = image.shape[0] // pool_size[0]
    pooled_width = image.shape[1] // pool_size[1]
    pooled_image = np.zeros((pooled_height, pooled_width))

    for i in range(pooled_height):
        for j in range(pooled_width):
            start_i, end_i = i * pool_size[0], (i + 1) * pool_size[0]
            start_j, end_j = j * pool_size[1], (j + 1) * pool_size[1]
            pooled_image[i, j] = np.mean(image[start_i:end_i, start_j:end_j])

    return pooled_image


# Step 3: Preprocess the data (e.g., scale features)
def preprocess_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled

In [2]:
# Step 4: Train a classifier (e.g., Logistic Regression)
def train_model(X_train, y_train):
    model = LogisticRegression(
        max_iter=1000, solver="lbfgs", multi_class="multinomial", random_state=42
    )
    model.fit(X_train, y_train)
    return model

In [3]:
# Step 5: Evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


# Step 6: Make predictions on new data
def predict(model, new_data):
    predictions = model.predict(new_data)
    return predictions

In [4]:
import tqdm

# Load and split the data
X, y = load_data()

pool_size = 7
X_pooled = np.zeros((X.shape[0], (28 // pool_size) * (28 // pool_size)))
for i in tqdm.tqdm(range(X.shape[0])):
    image = X[i].reshape(28, 28)
    pooled_image = avg_pooling(image, (pool_size, pool_size))
    X_pooled[i] = pooled_image.flatten()

X_train, X_test, y_train, y_test = split_data(X_pooled, y)

# Preprocess the data
X_train_scaled, X_test_scaled = preprocess_data(X_train, X_test)

100%|██████████| 70000/70000 [00:03<00:00, 17504.64it/s]


In [5]:
# Train the model
model = train_model(X_train_scaled, y_train)

/Users/robert/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [6]:
# Evaluate the model
evaluate_model(model, X_test_scaled, y_test)

Accuracy: 0.7457142857142857
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.68      0.74      1343
           1       0.82      0.90      0.86      1600
           2       0.82      0.75      0.79      1380
           3       0.79      0.77      0.78      1433
           4       0.64      0.69      0.66      1295
           5       0.68      0.61      0.64      1273
           6       0.84      0.87      0.86      1396
           7       0.77      0.81      0.79      1503
           8       0.64      0.69      0.66      1357
           9       0.64      0.64      0.64      1420

    accuracy                           0.75     14000
   macro avg       0.74      0.74      0.74     14000
weighted avg       0.75      0.75      0.75     14000

Confusion Matrix:
 [[ 916   10   23   23   19   77   23    4  237   11]
 [   0 1446    6   36   13   50    3   10   24   12]
 [  11   33 1038   66   60   11   81   12   54   14]
 [  22   5

In [7]:
# Example prediction
sample_data = X_test_scaled[:5]  # Predicting for the first 5 test samples
predictions = predict(model, sample_data)
print("Sample Predictions:", predictions)

Sample Predictions: [8 9 5 7 7]


In [24]:
import numpy as np


class LinUCB:
    def __init__(self, n_arms, n_features, alpha):
        self.alpha = alpha
        self.M = np.stack([np.eye(n_features) for _ in range(n_arms)])
        self.b = np.zeros((n_arms, n_features))  # Reward vector
        self.theta = np.random.randn(n_arms, n_features)  # Parameter estimate
        self.arms = n_arms

    def predict(self, feature_vectors):
        results = []
        for i in range(len(feature_vectors)):
            results.append(self.select_arm(feature_vectors[i]))
        return results

    def select_arm(self, x):
        p_values = []
        for k in range(self.arms):
            p = x.dot(self.theta[k]) + self.alpha * np.sqrt(
                x.T.dot(np.linalg.inv(self.M[k])).dot(x)
            )
            p_values.append(p)
        return np.argmax(p_values)

    def update(self, chosen_arm, reward, x):
        self.M[chosen_arm] += np.outer(x, x)
        self.b[chosen_arm] += reward * x
        self.theta[chosen_arm] = np.linalg.inv(self.M[chosen_arm]).dot(
            self.b[chosen_arm]
        )

In [25]:
# Example usage
n_features = X_train_scaled.shape[1]
alpha = 0.3
n_arms = 10  # Number of classes
linucb = LinUCB(n_arms, n_features, alpha)

In [26]:
for t in tqdm.tqdm(range(X_train_scaled.shape[0])):
    arm = linucb.select_arm(X_train_scaled[t])
    linucb.update(arm, arm == y_train[t], X_train_scaled[t])

100%|██████████| 56000/56000 [00:05<00:00, 9762.89it/s]


In [27]:
evaluate_model(linucb, X_test, y_test)

Accuracy: 0.5059285714285714
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.75      0.66      1343
           1       0.00      0.00      0.00      1600
           2       0.82      0.56      0.67      1380
           3       0.55      0.69      0.61      1433
           4       0.47      0.68      0.55      1295
           5       0.90      0.19      0.32      1273
           6       0.81      0.62      0.70      1396
           7       1.00      0.05      0.10      1503
           8       0.34      0.79      0.48      1357
           9       0.38      0.81      0.51      1420

    accuracy                           0.51     14000
   macro avg       0.58      0.52      0.46     14000
weighted avg       0.58      0.51      0.45     14000

Confusion Matrix:
 [[1010    0    2    3   24    0   21    0  248   35]
 [   1    0  111  305   61    6    1    0  823  292]
 [  64    0  775   65  147    1  123    0  167   38]
 [  79    

/Users/robert/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/robert/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/robert/miniconda3/envs/neural_bandits/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,